In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [59]:
df = pd.read_csv('bug-attraction.csv')

In [60]:
df.head()

,Light Type,Model Lamp,Location,Site,Standardized Moon,Standardized Mean Temp,Standardized Mean Humidity,Standardized Max Wind,% Moon Visible,Start Tem,End Temp,Mean Temp,Humid Start,Humid End,Mean Humidity,Max Wind,Illumination,Diptera,Hymenoptera,Hemiptera,Psocoptera,Coleoptera,Collembola,Arachnid,Thysanura,Isoptera,Lepidoptera,Neuroptera,Larave,Orthoptera,Unident,Total,Other Orders
0,A,3500 Custom,BG,B3,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,1675,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0
1,B,2700 Custom,BG,B4,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,1145,21.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,2.0
2,C,2700 Custom,BG,B2,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,1830,8.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,3.0
3,LED,LED,BG,B5,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,275,4.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,5.0
4,CFL,CFL,BG,B1,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,253,2.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0


In [61]:
pd.set_option('display.max_columns', None)


In [63]:
# Veri setindeki boş değerlerin toplamını, verilerin sayısına bölerek yüzdesini alıyoruz
nan_percentage = df.isna().sum() / df.count() * 100

# Boş verilerin toplam sayısını belirledik
nan_count = df.isna().sum()

# Boş verilerin toplamını ve yüzdelik halini tablo haline getirdik. Axis = 0 satırlar için, Axis = 1 sütunlar için
nan_table = pd.concat([nan_count, nan_percentage], axis=1)

# Tablonun sütunlarının isimlerini belirledik
nan_table.columns = ['Count', 'Percentage']
nan_table

,Count,Percentage
Light Type,0,0.000000
Model Lamp,0,0.000000
Location,0,0.000000
Site,0,0.000000
Standardized Moon,0,0.000000
Standardized Mean Temp,0,0.000000
Standardized Mean Humidity,0,0.000000
Standardized Max Wind,0,0.000000
% Moon Visible,0,0.000000
Start Tem,0,0.000000


In [64]:
unique_values = {col: df[col].unique() for col in df.columns}


for col, values in unique_values.items():
    print(f"Sütun '{col}' için eşsiz değerler: {values}")

Sütun 'Light Type' için eşsiz değerler: ['A' 'B' 'C' 'LED' 'CFL' 'No']
Sütun 'Model Lamp' için eşsiz değerler: ['3500 Custom' '2700 Custom' 'LED' 'CFL' 'No' 'LED2' 'CFL2']
Sütun 'Location' için eşsiz değerler: ['BG' 'Stunt' 'LK']
Sütun 'Site' için eşsiz değerler: ['B3' 'B4' 'B2' 'B5' 'B1' 'B6' 'S3' 'S2' 'S1' 'S6' 'S4' 'S5' 'L3' 'L2'
 'L1' 'L6' 'L4' 'L5']
Sütun 'Standardized Moon' için eşsiz değerler: [0.9243604  0.7753059  0.67630701 0.40934371 0.18353726 0.0756396
 0.         0.08453838 0.28809789 0.86318131 0.96996663 0.68743048
 0.19688543 0.10789766 0.30478309 0.52502781 0.61735261 0.72191324
 0.88431591 0.99888765 0.99443826 0.9621802  0.90656285 0.82091212
 0.61512792 0.49054505 0.35150167 0.2269188  0.32703003 0.53726363
 0.96329255 1.        ]
Sütun 'Standardized Mean Temp' için eşsiz değerler: [0.44871795 0.32051282 0.50961539 0.19230769 0.06410256 0.33653846
 0.23717949 0.25320513 0.29166667 0.3974359  0.5        0.34935897
 0.19871795 0.13461539 0.21794872 1.         0.26602

In [65]:
df = pd.get_dummies(df,columns=['Light Type','Model Lamp','Location','Site'],drop_first=True, dtype=int)

In [66]:
df.head()

,Standardized Moon,Standardized Mean Temp,Standardized Mean Humidity,Standardized Max Wind,% Moon Visible,Start Tem,End Temp,Mean Temp,Humid Start,Humid End,Mean Humidity,Max Wind,Illumination,Diptera,Hymenoptera,Hemiptera,Psocoptera,Coleoptera,Collembola,Arachnid,Thysanura,Isoptera,Lepidoptera,Neuroptera,Larave,Orthoptera,Unident,Total,Other Orders,Light Type_B,Light Type_C,Light Type_CFL,Light Type_LED,Light Type_No,Model Lamp_3500 Custom,Model Lamp_CFL,Model Lamp_CFL2,Model Lamp_LED,Model Lamp_LED2,Model Lamp_No,Location_LK,Location_Stunt,Site_B2,Site_B3,Site_B4,Site_B5,Site_B6,Site_L1,Site_L2,Site_L3,Site_L4,Site_L5,Site_L6,Site_S1,Site_S2,Site_S3,Site_S4,Site_S5,Site_S6
0,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,1675,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,1145,21.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,1830,8.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,3.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,275,4.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,5.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.92436,0.448718,0.628571,0.0,93.0,18.3,15.0,16.65,61,65,63.0,0.0,253,2.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
imputer = KNNImputer(n_neighbors=3)

In [76]:
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [87]:
y = df_imputed['Total']
x = df_imputed.drop(columns=['Total'])

In [104]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.5,random_state=10)

In [122]:
lr = LinearRegression()
model = lr.fit(x_train,y_train)
model.score(x_test,y_test)

1.0

In [105]:
rf = RandomForestRegressor(n_estimators=200)
model = rf.fit(x_train,y_train)
model.score(x_test,y_test)

0.9302688489954171